## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,48.33,77,7,4.34,clear sky
1,1,Zapolyarnyy,RU,69.4154,30.8136,36.68,79,2,4.72,clear sky
2,2,Atuona,PF,-9.8000,-139.0333,78.17,72,10,11.97,clear sky
3,3,Butaritari,KI,3.0707,172.7902,81.50,77,5,10.96,clear sky
4,4,Ulladulla,AU,-35.3500,150.4667,52.70,52,61,19.98,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Atuona,PF,-9.8000,-139.0333,78.17,72,10,11.97,clear sky
3,3,Butaritari,KI,3.0707,172.7902,81.50,77,5,10.96,clear sky
10,10,Olinda,BR,-8.0089,-34.8553,78.53,94,75,4.61,broken clouds
12,12,Georgetown,MY,5.4112,100.3354,76.95,88,40,0.00,scattered clouds
16,16,Akyab,MM,20.1500,92.9000,82.02,80,35,11.59,scattered clouds
17,17,Albany,US,42.6001,-73.9662,75.63,75,35,4.50,scattered clouds
19,19,Sao Filipe,CV,14.8961,-24.4956,75.54,78,5,13.22,clear sky
20,20,Rikitea,PF,-23.1203,-134.9692,77.45,76,94,12.57,overcast clouds
25,25,Pacific Grove,US,36.6177,-121.9166,75.54,64,0,8.01,clear sky
35,35,Lompoc,US,34.6391,-120.4579,76.69,62,0,32.01,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No empty rows found
clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.17,clear sky,-9.8000,-139.0333,
3,Butaritari,KI,81.50,clear sky,3.0707,172.7902,
10,Olinda,BR,78.53,broken clouds,-8.0089,-34.8553,
12,Georgetown,MY,76.95,scattered clouds,5.4112,100.3354,
16,Akyab,MM,82.02,scattered clouds,20.1500,92.9000,
17,Albany,US,75.63,scattered clouds,42.6001,-73.9662,
19,Sao Filipe,CV,75.54,clear sky,14.8961,-24.4956,
20,Rikitea,PF,77.45,overcast clouds,-23.1203,-134.9692,
25,Pacific Grove,US,75.54,clear sky,36.6177,-121.9166,
35,Lompoc,US,76.69,clear sky,34.6391,-120.4579,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
clean_hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
676,Caohe,CN,76.87,overcast clouds,30.2297,115.4335,Ai Jing Hotel
681,Kralendijk,BQ,80.58,clear sky,12.1500,-68.2667,Divi Flamingo Beach Resort & Casino
685,Half Moon Bay,US,82.80,clear sky,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
686,Celestun,MX,77.34,overcast clouds,20.8667,-90.4000,Posada Lilia
688,Upata,VE,79.20,broken clouds,8.0086,-62.3989,Posada Villa Nela
689,Marawi,PH,80.44,clear sky,7.9986,124.2928,Derogongan Residence
691,Payyannur,IN,78.10,overcast clouds,12.1000,75.2000,Kavvayi Beach House
693,Shingu,JP,77.40,overcast clouds,33.7333,135.9833,Shingu UI Hotel
694,Tecoanapa,MX,82.27,overcast clouds,16.5167,-98.7500,Hotel Manglares
695,Puerto Colombia,CO,80.55,overcast clouds,10.9878,-74.9547,Hotel El Emigrante


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.cvs"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))